In [1]:
from tensorflow import keras

In [4]:
import glob
train_path="csv/anomaly/*.csv"
files=glob.glob(train_path)
files

['csv/anomaly\\0.csv',
 'csv/anomaly\\2.csv',
 'csv/anomaly\\3.csv',
 'csv/anomaly\\4.csv',
 'csv/anomaly\\5.csv']

In [5]:
import pandas as pd
import numpy as np
dfs=[ pd.read_csv(name) for name in files]

In [6]:
dfs[3].columns

Index(['frame', 'time', 'blurriness', 'lighting_change', 'block', 'movement',
       'normal', 'has_blur', 'has_move', 'has_overexposure', 'has_block'],
      dtype='object')

In [72]:
Xs_train,ys_train=[],[]
X_val,y_val=None,None
labels=['normal', 'has_blur', 'has_move', 'has_overexposure', 'has_block']
for i,df in enumerate(dfs):
    print(df.isna().sum())
    if i==4-1:
        X_val,y_val=df.drop(labels,axis=1),df[labels]
    else:
        Xs_train.append(df.drop(labels,axis=1))
        ys_train.append(df[labels])
    

frame               0
time                0
blurriness          0
lighting_change     0
block               0
movement            0
normal              0
has_blur            0
has_move            0
has_overexposure    0
has_block           0
dtype: int64
frame               0
time                0
blurriness          0
lighting_change     0
block               0
movement            0
normal              0
has_blur            0
has_move            0
has_overexposure    0
has_block           0
dtype: int64
frame               0
time                0
blurriness          0
lighting_change     0
block               0
movement            0
normal              0
has_blur            0
has_move            0
has_overexposure    0
has_block           0
dtype: int64
frame               0
time                0
blurriness          0
lighting_change     0
block               0
movement            0
normal              0
has_blur            0
has_move            0
has_overexposure    0
has_block      

In [19]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [73]:
#Маштабирование
Xs_train_scaled=[]
Xs_train_scaled.append(scaler.fit_transform(Xs_train[0]))
Xs_train_scaled+=[scaler.transform(X) for X in Xs_train[1:]]

ys_train_np=[y.values for y in ys_train]

X_val_scaled,y_val_np=scaler.transform(X_val),y_val.values

In [107]:
def reshape_to_time_pockets(X,y,timesteps):
    print(X.shape,y.shape)
    n_packs=X.shape[0]//timesteps*timesteps
    X_take=X[:n_packs]
    y_take=y[:n_packs]
    print(X_take.shape,y_take.shape)
    return X_take.reshape(n_packs//timesteps,timesteps,X.shape[1]),y_take.reshape(n_packs//timesteps,timesteps,y.shape[1])

In [197]:
timesteps=1

X_val_timesteps,y_val_timesteps=reshape_to_time_pockets(X_val_scaled,y_val_np,timesteps)
Xs_train_timesteps,ys_train_timesteps=[],[]
for X_ts,y_ts in zip(Xs_train_scaled,ys_train_np):
    X,y=reshape_to_time_pockets(X_ts,y_ts,timesteps)
    Xs_train_timesteps.append(X)
    ys_train_timesteps.append(y)
    

(5062, 6) (5062, 5)
(5062, 6) (5062, 5)
(18822, 6) (18822, 5)
(18822, 6) (18822, 5)
(29838, 6) (29838, 5)
(29838, 6) (29838, 5)
(3518, 6) (3518, 5)
(3518, 6) (3518, 5)
(18088, 6) (18088, 5)
(18088, 6) (18088, 5)


In [207]:
#Норм 0 фокус 1 засвет 2 движение 3

model=keras.Sequential([
    keras.layers.LSTM(128,input_shape=(timesteps,6),activation="tanh",return_sequences=True),
    keras.layers.SimpleRNN(64,input_shape=(timesteps,6),activation="tanh",return_sequences=True),
    keras.layers.Dense(6,input_shape=(timesteps,6),activation="relu"),
    keras.layers.Dense(32,activation="relu"),
    keras.layers.Dropout(rate=0.7),
    keras.layers.Dense(16,activation="relu"),
    keras.layers.Dropout(rate=0.7),
    keras.layers.Dense(5,activation="sigmoid")
])
model.compile(loss="categorical_crossentropy",optimizer="sgd",metrics=["accuracy"])
model.summary()

Model: "sequential_51"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_77 (Dense)            (None, 1, 6)              42        
                                                                 
 dense_78 (Dense)            (None, 1, 32)             224       
                                                                 
 dropout_8 (Dropout)         (None, 1, 32)             0         
                                                                 
 dense_79 (Dense)            (None, 1, 16)             528       
                                                                 
 dropout_9 (Dropout)         (None, 1, 16)             0         
                                                                 
 dense_80 (Dense)            (None, 1, 5)              85        
                                                                 
Total params: 879 (3.43 KB)
Trainable params: 879 (3.

In [208]:
X_val_scaled.shape,y_val_np.shape

((5062, 6), (5062, 5))

In [209]:
for X_train,y_train in zip(Xs_train_timesteps,ys_train_timesteps):
    print(X_train.shape,y_train.shape)
    model.fit(X_train,y_train
              ,shuffle=False,
              validation_data=(X_val_timesteps,y_val_timesteps),batch_size=100,epochs=30)

(18822, 1, 6) (18822, 1, 5)
Epoch 1/30
189/189 [==============================] - 1s 3ms/step - loss: 0.8614 - accuracy: 0.8607 - val_loss: 3.4913 - val_accuracy: 0.4609
Epoch 2/30
189/189 [==============================] - 0s 2ms/step - loss: 0.4354 - accuracy: 0.9552 - val_loss: 4.5087 - val_accuracy: 0.4609
Epoch 3/30
189/189 [==============================] - 0s 2ms/step - loss: 0.3368 - accuracy: 0.9575 - val_loss: 5.0027 - val_accuracy: 0.4609
Epoch 4/30
189/189 [==============================] - 0s 2ms/step - loss: 0.2947 - accuracy: 0.9574 - val_loss: 5.2532 - val_accuracy: 0.4609
Epoch 5/30
189/189 [==============================] - 0s 2ms/step - loss: 0.2636 - accuracy: 0.9575 - val_loss: 5.5220 - val_accuracy: 0.4609
Epoch 6/30
189/189 [==============================] - 0s 2ms/step - loss: 0.2477 - accuracy: 0.9575 - val_loss: 5.7072 - val_accuracy: 0.4609
Epoch 7/30
189/189 [==============================] - 0s 2ms/step - loss: 0.2331 - accuracy: 0.9575 - val_loss: 5.9168 -

In [211]:
model.evaluate(X_val_timesteps,y_val_timesteps)

159/159 [==============================] - 0s 980us/step - loss: 1.8699 - accuracy: 0.4609


[1.869904637336731, 0.46088501811027527]